# Welcome to the MITO Dataset!

In this tutorial, we will show:
1. How to load mmWave images within the dataset
2. How to visualize mmWave images

***Before beginning this tutorial, please follow the installation steps listed in the README.***

In [ ]:
# Import dependencies
import sys
sys.path.append('../src')
from utils import utilities
from utils.object_information import ObjectInformation, ObjectAttributes
from utils.generic_loader import *
from utils.visualization import *

[//]: # (TODO: Does this belong here or elsewhere?)

### Understanding the Dataset Structure

Each object in our dataset contains a unique ID and name. These are the same as those defined in the YCB dataset (http://ycb-benchmarks.s3-website-us-east-1.amazonaws.com/). 

The data for an object is stored in a folder with the following naming convention: ```<object_id>_<object_name>```

Within this folder, our dataset is structured as follows:

[//]: # (ensp = 2, emsp = 4, nbsp = 1 space)

data  
&emsp;├── 001_chips_can  
&emsp; |&emsp;&emsp;├── robot_collected  
&emsp; |&emsp;&emsp; |&emsp;&emsp;└── 0_0_0  
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp;├── exp1  
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp; | &emsp;&emsp;├── los  
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp; | &emsp;&emsp; |&emsp;&emsp;├── processed  
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp; | &emsp;&emsp; |&emsp;&emsp; |&emsp;&emsp;├── camera&ensp;# This folder contains processed camera data such as segmentation masks  
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp; | &emsp;&emsp; |&emsp;&emsp; |&emsp;&emsp;└── radars&emsp;# This folder contains processed radar images  
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp; | &emsp;&emsp; |&emsp;&emsp;└── unprocessed  
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp; | &emsp;&emsp; |&emsp;&emsp; &emsp;&emsp;├── camera&ensp;# This folder contains raw camera images  
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp; | &emsp;&emsp; |&emsp;&emsp; &emsp;&emsp;└── radars&emsp;# This folder contains raw unprocessed radar measurements   
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp; | &emsp;&emsp;└── nlos  
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp; | &emsp;&emsp; &emsp;&emsp;└── ...   
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp;└── exp2  
&emsp; |&emsp;&emsp; |&emsp;&emsp; &emsp;&emsp;  &emsp;&emsp;└── ...  
&emsp; |&emsp;&emsp;└── simulation   
&emsp; |&emsp;&emsp; &emsp;&emsp;└── 0_0_0  
&emsp; |&emsp;&emsp; &emsp;&emsp; &emsp;&emsp;├── processed  
&emsp; |&emsp;&emsp; &emsp;&emsp; &emsp;&emsp; |&emsp;&emsp;└── ...  
&emsp; |&emsp;&emsp; &emsp;&emsp; &emsp;&emsp;└── unprocessed   
&emsp; |&emsp;&emsp; &emsp;&emsp; &emsp;&emsp;&ensp;&emsp;&emsp;└── ...  
&emsp;├── <object_id>_<object_name>  
&emsp;└── ...

# Loading mmWave Images

Next, we will load a sample mmWave image. To do so, our code follows 3 steps:
1. We define the parameters of the image we want to use. For example, do we want to load a simulation or real-world image? A line-of-sight or non-line-of-sight image?
2. We use object_information.py to load information about our requested object. For example, if we only have the name of the object, we can find the object ID with this class.
3. We use the generic_loader.py file, which is responsible for loading/saving all files within the dataset, to load the image.

In [ ]:
########################
# Step 1: Parameters to Change
obj_id = '052' # ID of object (including leading zeros). Note: One of obj_name or obj_id can be None, and the code will fill it in below
obj_name = 'extra_large_clamp' # Name of object. Note: One of obj_name or obj_id can be None, and the code will fill it in below
is_sim = False # Whether to use simulation (if True) or real-world (if False) data
is_specular_sim = False # Whether to plot specular simulation. If False, will plot edge simulation. This is ignored if is_sim is False
is_los = True # Whether to load line-of-sight (if True) or non-line-of-sight (if False). This parameter is ignored for simlation data
exp_num = '1' # Whether to load experiment 1 or 2. Most objects have data for both 
radar_type = '77_ghz' # Which radar type to load (either '77_ghz' or '24_ghz')
########################

# Define image_file_ext. This is the extension added to the end of the filename to ensure we are loading the correct file
if is_sim:
    if is_specular_sim:
        image_file_ext = utilities.load_param_json()['processing']['specular_extension']
    else:
        image_file_ext = utilities.load_param_json()['processing']['edges_extension']
else:
    image_file_ext = utilities.load_param_json()['processing']['robot_collected_extension']

# Step 2: Load necessary information about object
# This information is loaded from the experiments.csv and objects.csv files for you
obj_info = ObjectInformation()
obj_id, obj_name = obj_info.fill_in_identifier_sep(obj_name, obj_id) # Fill in empty name or ID
crop = obj_info.get_object_info(ExperimentAttributes.CROP, obj_id=obj_id, name=obj_name, exp_num=exp_num) # Whether to crop image when loading it
crop_high = obj_info.get_object_info(ExperimentAttributes.CROP_HIGH, obj_id=obj_id, name=obj_name, exp_num=exp_num) # Whether to crop image at a higher point (due to different physical setup)
attr = ExperimentAttributes.LOS_BACKGROUND_ID if is_los else ExperimentAttributes.NLOS_BACKGROUND_ID
background_exp = None if is_sim or radar_type=='77_ghz' else obj_info.get_object_info(attr, obj_id=obj_id, name=obj_name, exp_num=exp_num)

# Create a GenericLoader object
loader = GenericLoader(obj_id, obj_name, is_sim=is_sim, is_los=is_los, exp_num=exp_num)

# Step 3: Load the image
image, (x_locs, y_locs, z_locs), antenna_locs = loader.load_image_file(radar_type, background_subtraction=background_exp, ext=image_file_ext, crop=crop, crop_high=crop_high)

Next, let us better understand these image outputs. Each mmWave image file contains the following information:
- image: This is the mmWave image, which is stored as a 3D, complex numpy array with shape (X,Y,Z). 
- locs: These define the coordinates of each voxel within the mmWave image. This will be a tuple of numpy arrays:
    - x_locs will be the coordinates of each voxel in the x_dimension. It is a numpy array with size (X, )
    - y_locs will be the coordinates of each voxel in the y_dimension. It is a numpy array with size (Y, )
    - z_locs will be the coordinates of each voxel in the z_dimension. It is a numpy array with size (Z, )
- antenna_locs: These are the locations at which the robot took each measurement to form the mmWave image. 

Therefore, the 3D coordinate of the image at voxel (i,j,k) would be [x_locs[i], y_locs[j], z_locs[k]]

[//]: # (TODO: This would be easier to visualize with an image)

For example, for this object, we can use the following code to learn more about the outputs:

In [ ]:
print(f'Image shape: {image.shape}')
print(f'x_locs shape: {x_locs.shape}')
print(f'y_locs shape: {y_locs.shape}')
print(f'z_locs shape: {z_locs.shape}')

voxel_idx = [0, 0, 0]
voxel_coord = [x_locs[voxel_idx[0]], y_locs[voxel_idx[1]], z_locs[voxel_idx[2]]]
print(f'The coordinate of voxel {voxel_idx} would be {voxel_coord}')

# Visualizing mmWave Images

Finally, let us visualize a mmWave image. We will do so using the visualization.py file. 

To visualize the 3D, complex-valued mmWave images, our plotter:
1. Takes the magnitude of the complex values
2. Projects the mmWave image along a given dimension (by averaging the values). We typically project along the vertical (z) dimension.

In [ ]:
########################
# Parameters to Change
projection_dim = 2 # The dimension along which to project the image. We typically plot along the vertical=z=2 dimension
plot_all_slices = False # By default, we show a single 2D image which is the average of all values along the projection_dim. If you set this to True, you can see the full 3D image plotted as a series of 2D images (taken as slices along the projection_dim)
########################

# Create visualizer object
visualizer = Visualizer()

# Plot the SAR image. This function will project the 3D image to 2D, and plot the absolute value
visualizer.plot_sar_image(image, x_locs, y_locs, z_locs, plot_dim=projection_dim, normalization=None, plot_all_slices=plot_all_slices, obj_name=obj_name, title=f'2D SAR image of {obj_name}')